# Load ngữ liệu

Chuẩn bị đường dẫn lưu preprocessing

In [1]:
import pandas as pd
import os

# Đường dẫn thư mục dữ liệu gốc và thư mục lưu dữ liệu sạch
INPUT_DIR = "./UIT-ViHSD"
OUTPUT_DIR = "./UIT-ViHSD-preprocessed"

# Tạo thư mục output nếu chưa tồn tại
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Đã tạo thư mục: {OUTPUT_DIR}")

print("Sẵn sàng xử lý dữ liệu.")

Sẵn sàng xử lý dữ liệu.


In [2]:
# Đọc dữ liệu
try:
    df_train = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
    df_dev = pd.read_csv(os.path.join(INPUT_DIR, "dev.csv"))
    df_test = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"))
    
    print(f"Train shape gốc: {df_train.shape}")
    print(f"Dev shape gốc:   {df_dev.shape}")
    print(f"Test shape gốc:  {df_test.shape}")
except FileNotFoundError:
    print("❌ Lỗi: Không tìm thấy file dữ liệu. Hãy kiểm tra lại đường dẫn.")

Train shape gốc: (24048, 2)
Dev shape gốc:   (2672, 2)
Test shape gốc:  (6680, 2)


# 01. Word-segmentating texts into words by the pyvi tool8

Xử lý sơ bộ & Định nghĩa hàm Tách từ

In [3]:
from pyvi import ViTokenizer
# 1. Xử lý Missing Values trước để tránh lỗi khi tách từ
def drop_missing(df, name):
    initial = len(df)
    df = df.dropna(subset=['free_text'])
    # Chuyển về string
    df['free_text'] = df['free_text'].astype(str) 
    print(f"- {name}: Đã loại bỏ {initial - len(df)} dòng Null.")
    return df

df_train = drop_missing(df_train, "Train")
df_dev = drop_missing(df_dev, "Dev")
df_test = drop_missing(df_test, "Test")

# 2. Định nghĩa hàm tách từ an toàn
def safe_segmentation(text):
    try:
        # PyVi thực hiện tách từ: "sinh viên" -> "sinh_viên"
        return ViTokenizer.tokenize(text)
    except Exception as e:
        return text # Trả về text gốc nếu lỗi

print("\n✅ Đã chuẩn bị xong hàm xử lý.")

- Train: Đã loại bỏ 2 dòng Null.
- Dev: Đã loại bỏ 0 dòng Null.
- Test: Đã loại bỏ 0 dòng Null.

✅ Đã chuẩn bị xong hàm xử lý.


C:\Users\Admin\AppData\Local\Temp\ipykernel_12836\2941027614.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['free_text'] = df['free_text'].astype(str)


Thực thi Tách từ (Tokenization)

In [4]:
print("⏳ Đang tách từ tập TRAIN (vui lòng đợi)...")
df_train['free_text'] = df_train['free_text'].apply(safe_segmentation)

print("⏳ Đang tách từ tập DEV...")
df_dev['free_text'] = df_dev['free_text'].apply(safe_segmentation)

print("⏳ Đang tách từ tập TEST...")
df_test['free_text'] = df_test['free_text'].apply(safe_segmentation)

print("✅ Hoàn tất tách từ.")

⏳ Đang tách từ tập TRAIN (vui lòng đợi)...


⏳ Đang tách từ tập DEV...
⏳ Đang tách từ tập TEST...
✅ Hoàn tất tách từ.


Lưu kết quả vào UIT-ViHSD-preprocessed

In [5]:
# Định nghĩa tên file đầu ra
out_train = os.path.join(OUTPUT_DIR, "train_segmented.csv")
out_dev = os.path.join(OUTPUT_DIR, "dev_segmented.csv")
out_test = os.path.join(OUTPUT_DIR, "test_segmented.csv")

# Lưu file
df_train.to_csv(out_train, index=False)
df_dev.to_csv(out_dev, index=False)
df_test.to_csv(out_test, index=False)

print(f"✅ Đã lưu dữ liệu vào: {OUTPUT_DIR}")
print(f"   1. {out_train}")
print(f"   2. {out_dev}")
print(f"   3. {out_test}")

✅ Đã lưu dữ liệu vào: ./UIT-ViHSD-preprocessed
   1. ./UIT-ViHSD-preprocessed\train_segmented.csv
   2. ./UIT-ViHSD-preprocessed\dev_segmented.csv
   3. ./UIT-ViHSD-preprocessed\test_segmented.csv


Kiểm tra lại dữ liệu (Sanity Check)

In [6]:
print("--- Mẫu dữ liệu sau khi tách từ (Train) ---")
# Hiển thị cột text rộng hơn để dễ nhìn
pd.set_option('display.max_colwidth', 150) 
print(df_train[['free_text', 'label_id']].head(5))

--- Mẫu dữ liệu sau khi tách từ (Train) ---
                                                                                                          free_text  \
0                 Em được làm fan cứng luôn rồi nè ❤ ️ reaction quá hay quá cute coi mấy giờ này quá hợp_lí = ] ] ]   
1  Đúng là bọn mắt híp lò_xo thụt : ) ) ) bên việt_nam t cái này ra cách đây 10 năm r và bọn t gọi là cái L : ) ) )   
2                                                                          Đậu_Văn Cường giờ giống thằng sida hơn à   
3                                                  CÔN_ĐỒ CỤC_SÚC VÔ NHÂN_TÍNH ĐỀ NGHI VN. NHÀ_NƯỚC VN BAN THƯỞNG .   
4                                                           Từ lý_thuyết đến thực_hành là cả 1 câu_chuyện dài = ) )   

   label_id  
0         0  
1         2  
2         0  
3         2  
4         0  


So sánh với dữ liệu cũ

In [7]:
import pandas as pd
import os

# --- CẤU HÌNH ---
RAW_FILE = "./UIT-ViHSD/train.csv"
SEG_FILE = "./UIT-ViHSD-preprocessed/train_segmented.csv"

def compare_segmentation(n_samples=10):
    print("👀 Đang lấy mẫu để so sánh...")
    
    # 1. Đọc dữ liệu
    try:
        df_raw = pd.read_csv(RAW_FILE)
        df_seg = pd.read_csv(SEG_FILE)
    except FileNotFoundError:
        print("❌ Không tìm thấy file. Hãy chắc chắn bạn đã chạy các bước trước đó.")
        return

    # 2. Đồng bộ hóa dữ liệu (vì file segmented đã bị drop Null)
    # Ta cần drop Null ở file Raw tương tự để index khớp nhau
    df_raw = df_raw.dropna(subset=['free_text']).reset_index(drop=True)
    df_seg = df_seg.reset_index(drop=True)
    
    # 3. Lấy mẫu ngẫu nhiên
    # Chọn các câu có độ dài trung bình (để dễ nhìn)
    # df_seg['len'] = df_seg['free_text'].str.len()
    # sample_indices = df_seg[(df_seg['len'] > 50) & (df_seg['len'] < 150)].sample(n_samples).index
    
    # Hoặc lấy ngẫu nhiên hoàn toàn
    if len(df_seg) > n_samples:
        sample_indices = df_seg.sample(n_samples, random_state=42).index
    else:
        sample_indices = df_seg.index

    # 4. Tạo bảng so sánh
    comparison_df = pd.DataFrame({
        'ORIGINAL (Gốc)': df_raw.loc[sample_indices, 'free_text'].values,
        'SEGMENTED (Đã tách từ)': df_seg.loc[sample_indices, 'free_text'].values,
        'Label': df_seg.loc[sample_indices, 'label_id'].values
    })
    
    # 5. Hiển thị đẹp
    pd.set_option('display.max_colwidth', None) # Hiển thị toàn bộ nội dung
    display(comparison_df) # Dùng display() trong Jupyter để đẹp hơn print()

# Gọi hàm
compare_segmentation(10)

👀 Đang lấy mẫu để so sánh...


,ORIGINAL (Gốc),SEGMENTED (Đã tách từ),Label
0,Luân TRrần chớ k phải là bắn pub hả yêu chi khổ.. a lo ổm định đi r cưới luôn khỏi yêu cho tk e này chiếc ex 😂😂,Luân TRrần chớ k phải là bắn pub hả yêu chi khổ . . a lo ổm định đi r cưới luôn khỏi yêu cho tk e này chiếc ex 😂 😂,0
1,Thằng,Thằng,0
2,Người ieu cua ai bắt về dùm đi cảm ơn ...cho nó hút khí bot dien,Người ieu cua ai bắt về dùm đi cảm_ơn ... cho nó hút khí bot dien,0
3,Biết thế là đc r. Dẫu sao cũng nhắc nhở a e là. Đọc thôi cái gì cần nói hãy nói. Cái thằng admin kia nó k hiểu vđề gì phán ng này là chó ng kia là chó. Nó lại hiểu đc tiếng chó :-?...,Biết thế_là đc r . Dẫu_sao cũng nhắc_nhở a e là . Đọc thôi cái gì cần nói hãy nói . Cái thằng admin kia nó k hiểu vđề gì phán ng này là chó ng kia là chó . Nó lại hiểu đc tiếng chó : - ? ...,2
4,Tú Nguyễn nước hoa mà tính vài trăm ngàn thì lấy đâu ra loại tốt b.,Tú Nguyễn nước_hoa mà tính vài trăm ngàn thì lấy đâu ra loại tốt b .,0
5,Còn ban sáng họp giao ban của CỤC CỨC,Còn ban sáng họp giao_ban của CỤC CỨC,1
6,Đm bọn thuyết phục xl,Đm bọn thuyết_phục xl,2
7,Thanh Lý Sim số đẹp--Giao dịch toàn quốc,Thanh_Lý_Sim số đẹp - - Giao_dịch toàn_quốc,0
8,Sting vô đầu,Sting vô đầu,0
9,Cảm thấy chửi lđao là 1 thú vui xả stress do bọn nó quá ngu và nguy hiểm :))),Cảm_thấy chửi lđao là 1 thú_vui xả stress do bọn nó quá ngu và nguy_hiểm : ) ) ),2


# 02. Loại bỏ stopword

Khai báo đường dẫn và Load Stopwords

In [8]:
import pandas as pd
import os

# --- CẤU HÌNH ĐƯỜNG DẪN ---
INPUT_DIR = "./UIT-ViHSD-preprocessed" # Lấy dữ liệu từ bước trước
STOPWORD_FILE = "vietnamese-stopwords.txt" # File stopwords của bạn

# --- HÀM LOAD STOPWORDS ---
def load_stopwords(filepath):
    stopwords = set()
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    # QUAN TRỌNG: Chuyển stopword thành dạng segmented (thay dấu cách bằng _)
                    # Ví dụ: "mặc dù" -> "mặc_dù" để khớp với dữ liệu đã tách từ
                    word_segmented = word.replace(' ', '_')
                    stopwords.add(word_segmented)
        print(f"✅ Đã load {len(stopwords)} từ dừng (stopwords).")
        return stopwords
    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy file {filepath}. Vui lòng kiểm tra lại.")
        return set()

# Load stopwords vào bộ nhớ
stopword_set = load_stopwords(STOPWORD_FILE)

# Kiểm tra thử vài từ
if len(stopword_set) > 0:
    print(f"Ví dụ vài từ trong set: {list(stopword_set)[:10]}")

✅ Đã load 1942 từ dừng (stopwords).
Ví dụ vài từ trong set: ['bằng_như', 'gì_đó', 'tới_mức', 'nhà_chung', 'chắc_chắn', 'quan_trọng', 'càng', 'đầu_tiên', 'thi_thoảng', 'làm_gì']


In [9]:
# Đọc các file đã segmented
try:
    df_train = pd.read_csv(os.path.join(INPUT_DIR, "train_segmented.csv"))
    df_dev = pd.read_csv(os.path.join(INPUT_DIR, "dev_segmented.csv"))
    df_test = pd.read_csv(os.path.join(INPUT_DIR, "test_segmented.csv"))
    
    # Xử lý NaN nếu có (do quá trình đọc ghi file)
    df_train['free_text'] = df_train['free_text'].fillna("").astype(str)
    df_dev['free_text'] = df_dev['free_text'].fillna("").astype(str)
    df_test['free_text'] = df_test['free_text'].fillna("").astype(str)

    print("✅ Đã load dữ liệu segmented thành công.")
except FileNotFoundError:
    print("❌ Không tìm thấy file dữ liệu segmented. Hãy chạy bước tách từ trước.")

✅ Đã load dữ liệu segmented thành công.


In [10]:
def remove_stopwords_from_text(text, stopword_set):
    """
    Input: "Hôm_nay tôi đi học"
    Stopwords: {"tôi", "đi"}
    Output: "Hôm_nay học"
    """
    if not text:
        return ""
    
    # Tách câu thành list các token (dựa vào khoảng trắng)
    tokens = text.split()
    
    # Lọc bỏ token nếu nó nằm trong stopword_set
    clean_tokens = [t for t in tokens if t.lower() not in stopword_set]
    
    # Nối lại thành chuỗi
    return ' '.join(clean_tokens)

print("✅ Đã định nghĩa hàm remove_stopwords.")

✅ Đã định nghĩa hàm remove_stopwords.


In [11]:
if len(stopword_set) > 0:
    print("⏳ Đang loại bỏ stopwords cho tập Train...")
    df_train['free_text'] = df_train['free_text'].apply(lambda x: remove_stopwords_from_text(x, stopword_set))

    print("⏳ Đang loại bỏ stopwords cho tập Dev...")
    df_dev['free_text'] = df_dev['free_text'].apply(lambda x: remove_stopwords_from_text(x, stopword_set))

    print("⏳ Đang loại bỏ stopwords cho tập Test...")
    df_test['free_text'] = df_test['free_text'].apply(lambda x: remove_stopwords_from_text(x, stopword_set))
    
    print("✅ Hoàn tất.")
else:
    print("⚠️ Cảnh báo: Tập stopword rỗng, bỏ qua bước này.")

⏳ Đang loại bỏ stopwords cho tập Train...
⏳ Đang loại bỏ stopwords cho tập Dev...
⏳ Đang loại bỏ stopwords cho tập Test...
✅ Hoàn tất.


In [12]:
# Kiểm tra kết quả
print("--- Mẫu dữ liệu sau khi bỏ Stopwords (Train) ---")
pd.set_option('display.max_colwidth', 150)
print(df_train[['free_text', 'label_id']].head(5))

# Lưu file
out_train = os.path.join(INPUT_DIR, "train_no_sw.csv")
out_dev = os.path.join(INPUT_DIR, "dev_no_sw.csv")
out_test = os.path.join(INPUT_DIR, "test_no_sw.csv")

df_train.to_csv(out_train, index=False)
df_dev.to_csv(out_dev, index=False)
df_test.to_csv(out_test, index=False)

print(f"\n✅ Đã lưu dữ liệu sạch vào folder {INPUT_DIR}:")
print(f"   - {out_train}")
print(f"   - {out_dev}")
print(f"   - {out_test}")

--- Mẫu dữ liệu sau khi bỏ Stopwords (Train) ---
                                                            free_text  \
0                fan cứng nè ❤ ️ reaction cute coi mấy hợp_lí = ] ] ]   
1  bọn mắt híp lò_xo thụt : ) ) ) việt_nam t 10 r bọn t gọi L : ) ) )   
2                                            Đậu_Văn Cường thằng sida   
3    CÔN_ĐỒ CỤC_SÚC VÔ NHÂN_TÍNH ĐỀ NGHI VN. NHÀ_NƯỚC VN BAN THƯỞNG .   
4                              lý_thuyết thực_hành 1 câu_chuyện = ) )   

   label_id  
0         0  
1         2  
2         0  
3         2  
4         0  

✅ Đã lưu dữ liệu sạch vào folder ./UIT-ViHSD-preprocessed:
   - ./UIT-ViHSD-preprocessed\train_no_sw.csv
   - ./UIT-ViHSD-preprocessed\dev_no_sw.csv
   - ./UIT-ViHSD-preprocessed\test_no_sw.csv


Kiểm tra dữ liệu trước và sau khi bỏ stopwords

In [13]:
import pandas as pd
import os

# --- CẤU HÌNH ---
INPUT_FILE = "./UIT-ViHSD-preprocessed/train_segmented.csv"

# --- HÀM XỬ LÝ ---
def filter_impacted_rows(input_path, stopword_set, n_limit=20):
    print(f"📂 Đang đọc dữ liệu từ: {input_path}")
    
    try:
        # Load dữ liệu gốc (đã tách từ nhưng chưa xóa stopword)
        df = pd.read_csv(input_path)
        df['free_text'] = df['free_text'].fillna("").astype(str)
        
        # Danh sách chứa kết quả
        impacted_rows = []
        
        for idx, row in df.iterrows():
            text = row['free_text']
            tokens = text.split()
            
            # Tìm các từ bị xóa
            removed = [t for t in tokens if t.lower() in stopword_set]
            
            # Chỉ lấy những dòng CÓ từ bị xóa (removed list không rỗng)
            if len(removed) > 0:
                # Tạo lại câu đã clean để so sánh
                cleaned = [t for t in tokens if t.lower() not in stopword_set]
                
                impacted_rows.append({
                    'Original (Segmented)': text,
                    'Cleaned': " ".join(cleaned),
                    '❌ REMOVED WORDS': ", ".join(set(removed)), # Set để loại bỏ từ trùng trong cùng 1 câu
                    'Label': row['label_id']
                })
                
            # Dừng nếu đã lấy đủ số lượng mẫu cần xem (để tiết kiệm thời gian chạy demo)
            if len(impacted_rows) >= n_limit:
                break
        
        # Tạo DataFrame hiển thị
        df_impact = pd.DataFrame(impacted_rows)
        
        print(f"✅ Đã tìm thấy {len(df_impact)} mẫu có sự thay đổi.")
        
        # Hiển thị
        pd.set_option('display.max_colwidth', None)
        display(df_impact)
        
    except FileNotFoundError:
        print("❌ Không tìm thấy file segmented. Vui lòng kiểm tra lại đường dẫn.")

# --- THỰC THI ---
# Giả định biến stopword_set đã có từ bước trước. 
# Nếu chưa có, bạn cần chạy lại Cell load stopwords.
if 'stopword_set' in globals() and len(stopword_set) > 0:
    # Lấy 20 mẫu có thay đổi để kiểm tra
    filter_impacted_rows(INPUT_FILE, stopword_set, n_limit=20)
else:
    print("⚠️ Biến 'stopword_set' chưa được khởi tạo. Vui lòng chạy lại Cell load stopwords ở trên.")

📂 Đang đọc dữ liệu từ: ./UIT-ViHSD-preprocessed/train_segmented.csv
✅ Đã tìm thấy 20 mẫu có sự thay đổi.


,Original (Segmented),Cleaned,❌ REMOVED WORDS,Label
0,Em được làm fan cứng luôn rồi nè ❤ ️ reaction quá hay quá cute coi mấy giờ này quá hợp_lí = ] ] ],fan cứng nè ❤ ️ reaction cute coi mấy hợp_lí = ] ] ],"giờ, hay, rồi, này, quá, Em, làm, luôn, được",0
1,Đúng là bọn mắt híp lò_xo thụt : ) ) ) bên việt_nam t cái này ra cách đây 10 năm r và bọn t gọi là cái L : ) ) ),bọn mắt híp lò_xo thụt : ) ) ) việt_nam t 10 r bọn t gọi L : ) ) ),"bên, cách, là, cái, đây, này, năm, và, Đúng, ra",2
2,Đậu_Văn Cường giờ giống thằng sida hơn à,Đậu_Văn Cường thằng sida,"giống, à, giờ, hơn",0
3,Từ lý_thuyết đến thực_hành là cả 1 câu_chuyện dài = ) ),lý_thuyết thực_hành 1 câu_chuyện = ) ),"là, cả, Từ, dài, đến",0
4,Đố chúng m nhận ra ai,Đố m,"chúng, ra, nhận, ai",0
5,Lúp lúp như chó .,Lúp lúp chó .,như,1
6,"Thế_mà mình nói mấy thằng bắc kì , bọn đó lại bâu vô nói mình phân_biệt ! : D","mấy thằng bắc kì , bọn bâu vô phân_biệt ! : D","Thế_mà, mình, lại, đó, nói",2
7,"Được anh ưi , lâu rồi không nghe phúc du rap 🤣","ưi , phúc du rap 🤣","Được, rồi, nghe, anh, lâu, không",0
8,Cắt cho trẻ trâu bớt thui mà 😂 😂 😂,Cắt trẻ trâu bớt thui 😂 😂 😂,"cho, mà",0
9,Nghe rất vô_lí nhưng cực_kì thuyết_phục,vô_lí cực_kì thuyết_phục,"Nghe, nhưng, rất",0


# 3. Changing all texts into lower cases10

In [14]:
import pandas as pd
import os

# --- CẤU HÌNH ---
DIR = "./UIT-ViHSD-preprocessed"

# Đuôi file đầu vào (Chọn loại file bạn muốn xử lý tiếp)
# Nếu bạn ĐÃ chạy bỏ stopwords -> dùng "_no_sw.csv"
# Nếu bạn KHÔNG chạy bỏ stopwords -> dùng "_segmented.csv"
INPUT_SUFFIX = "_no_sw.csv" 

# Đuôi file đầu ra
OUTPUT_SUFFIX = "_lower.csv"

# --- LOAD DATA ---
try:
    print(f"📂 Đang đọc dữ liệu có đuôi '{INPUT_SUFFIX}'...")
    df_train = pd.read_csv(os.path.join(DIR, f"train{INPUT_SUFFIX}"))
    df_dev = pd.read_csv(os.path.join(DIR, f"dev{INPUT_SUFFIX}"))
    df_test = pd.read_csv(os.path.join(DIR, f"test{INPUT_SUFFIX}"))
    
    # Xử lý NaN phòng hờ
    df_train['free_text'] = df_train['free_text'].fillna("").astype(str)
    df_dev['free_text'] = df_dev['free_text'].fillna("").astype(str)
    df_test['free_text'] = df_test['free_text'].fillna("").astype(str)
    
    print("✅ Đã load xong 3 tập dữ liệu.")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file với đuôi {INPUT_SUFFIX}. Hãy kiểm tra lại folder {DIR}.")

📂 Đang đọc dữ liệu có đuôi '_no_sw.csv'...
✅ Đã load xong 3 tập dữ liệu.


In [15]:
def to_lowercase(text):
    return text.lower()

print("⏳ Đang chuyển đổi sang chữ thường...")

# Áp dụng
df_train['free_text'] = df_train['free_text'].apply(to_lowercase)
df_dev['free_text'] = df_dev['free_text'].apply(to_lowercase)
df_test['free_text'] = df_test['free_text'].apply(to_lowercase)

print("✅ Hoàn tất chuyển đổi.")

⏳ Đang chuyển đổi sang chữ thường...
✅ Hoàn tất chuyển đổi.


In [16]:
# Định nghĩa tên file
out_train = os.path.join(DIR, f"train{OUTPUT_SUFFIX}")
out_dev = os.path.join(DIR, f"dev{OUTPUT_SUFFIX}")
out_test = os.path.join(DIR, f"test{OUTPUT_SUFFIX}")

# Lưu file
df_train.to_csv(out_train, index=False)
df_dev.to_csv(out_dev, index=False)
df_test.to_csv(out_test, index=False)

print(f"✅ Đã lưu dữ liệu lowercase vào: {DIR}")
print(f"   - {out_train}")
print(f"   - {out_dev}")
print(f"   - {out_test}")

✅ Đã lưu dữ liệu lowercase vào: ./UIT-ViHSD-preprocessed
   - ./UIT-ViHSD-preprocessed\train_lower.csv
   - ./UIT-ViHSD-preprocessed\dev_lower.csv
   - ./UIT-ViHSD-preprocessed\test_lower.csv


In [17]:
# Lấy mẫu ngẫu nhiên để kiểm tra
print("--- Mẫu dữ liệu sau khi Lowercase ---")
pd.set_option('display.max_colwidth', 150)
print(df_train[['free_text', 'label_id']].sample(5))

# Kiểm tra kỹ thuật: Đếm số ký tự in hoa còn sót lại
upper_count = df_train['free_text'].str.count(r'[A-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝĂĐĨŨƠƯẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪỬỮỰỲỴỶỸ]').sum()
print(f"\nTổng số ký tự in hoa còn sót lại trong tập Train: {upper_count} (Kỳ vọng: 0)")

--- Mẫu dữ liệu sau khi Lowercase ---
                                               free_text  label_id
15159                                            hat wua         0
15572      minh nhật_éo chết dịch chết nổ đt 🤣 🤣 🤣 🤣 🤣 🤣         1
9080   tui tưởng bọn học_trò ghép clip ai_ngờ giải thiệt         0
22895                                                chó         0
19011                                       12 : 00 chửi         0

Tổng số ký tự in hoa còn sót lại trong tập Train: 0 (Kỳ vọng: 0)


# 4. Loại bỏ các đặc trưng Regex (Emoji, URL, Hashtag, Mention)

In [18]:
import pandas as pd
import re
import os

# --- CẤU HÌNH ---
DIR = "./UIT-ViHSD-preprocessed"
# File đầu vào: Lấy từ bước trước (lower case)
INPUT_SUFFIX = "_lower.csv" 

# --- HÀM LÀM SẠCH NÂNG CAO ---
def clean_special_patterns(text):
    if not isinstance(text, str):
        return str(text)
    
    # 1. Xử lý URL
    # Regex bắt http/https/www
    text = re.sub(r'http\S+|www\.\S+', '', text)
    
    # 2. Xử lý Mention (@Username)
    # Bắt @ theo sau là các ký tự không phải khoảng trắng
    text = re.sub(r'@\S+', ' ', text)
    
    # 3. Xử lý Hashtag (#Hashtag)
    # Cách 1: Xóa hẳn -> re.sub(r'#\S+', '', text)
    # Cách 2: Chỉ xóa dấu #, giữ lại chữ -> text.replace('#', '')
    # Dựa trên thống kê của bạn (#error, #name), tôi chọn xóa dấu # để giữ lại nội dung chữ nếu có nghĩa
    text = text.replace('#', '')
    
    # 4. Xử lý các ký tự đặc biệt khác (Dấu câu thừa, icon, emoji...)
    # Giữ lại: Chữ cái (bao gồm tiếng Việt), số, và các dấu câu cơ bản
    # Pattern này giữ lại: 
    # - \w: Chữ và số
    # - \s: Khoảng trắng
    # - Các dấu câu phổ biến: .,?!:;""''
    # Các ký tự lạ (emoji, symbol đặc biệt) sẽ bị loại bỏ
    # Lưu ý: Python regex mặc định \w chưa chắc bắt hết unicode tiếng Việt nếu không cấu hình kỹ.
    # Nên dùng cách phủ định: Xóa những gì KHÔNG PHẢI là word hoặc space hoặc punctuation
    
    # Tuy nhiên, đơn giản nhất là dùng regex thay thế các ký tự không mong muốn
    # Ở đây tôi sẽ xóa emoji và các ký tự symbol lạ, chỉ giữ lại text và dấu câu cơ bản
    # pattern = r'[^\w\s,.?!:;"\']' # Giữ lại word, space và dấu câu cơ bản
    # text = re.sub(pattern, ' ', text)
    
    # Xử lý khoảng trắng thừa (do việc xóa tạo ra)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# --- LOAD DỮ LIỆU ---
try:
    print("⏳ Đang đọc dữ liệu từ bước Lowercase...")
    df_train = pd.read_csv(os.path.join(DIR, f"train{INPUT_SUFFIX}"))
    df_dev = pd.read_csv(os.path.join(DIR, f"dev{INPUT_SUFFIX}"))
    df_test = pd.read_csv(os.path.join(DIR, f"test{INPUT_SUFFIX}"))
    
    # Fillna
    df_train['free_text'] = df_train['free_text'].fillna("").astype(str)
    df_dev['free_text'] = df_dev['free_text'].fillna("").astype(str)
    df_test['free_text'] = df_test['free_text'].fillna("").astype(str)

except FileNotFoundError:
    print("❌ Lỗi: Không tìm thấy file input. Vui lòng kiểm tra lại bước trước.")

# --- ÁP DỤNG LÀM SẠCH ---
print("⏳ Đang loại bỏ URL, Mention, Hashtag và ký tự đặc biệt...")

df_train['free_text'] = df_train['free_text'].apply(clean_special_patterns)
df_dev['free_text'] = df_dev['free_text'].apply(clean_special_patterns)
df_test['free_text'] = df_test['free_text'].apply(clean_special_patterns)

# --- XỬ LÝ LẠI NẾU CÓ DÒNG RỖNG ---
# Sau khi xóa hết đặc biệt, có thể có dòng thành rỗng
print("⏳ Kiểm tra dòng rỗng sau khi clean...")
empty_train = df_train[df_train['free_text'] == ''].shape[0]
if empty_train > 0:
    print(f"⚠️ Cảnh báo: Có {empty_train} dòng trở nên rỗng ở tập Train. (Sẽ giữ nguyên hoặc điền 'empty')")
    # Tùy chọn: df_train = df_train[df_train['free_text'] != '']
    # Nhưng để an toàn cho việc khớp dòng, ta nên giữ nguyên hoặc điền placeholder
    df_train.loc[df_train['free_text'] == '', 'free_text'] = 'empty_content' 
    df_dev.loc[df_dev['free_text'] == '', 'free_text'] = 'empty_content'
    df_test.loc[df_test['free_text'] == '', 'free_text'] = 'empty_content'

# --- LƯU FILE CUỐI CÙNG (FINAL) ---
# Tên file chuẩn: train.csv, dev.csv, test.csv
out_train = os.path.join(DIR, "train.csv")
out_dev = os.path.join(DIR, "dev.csv")
out_test = os.path.join(DIR, "test.csv")

df_train.to_csv(out_train, index=False)
df_dev.to_csv(out_dev, index=False)
df_test.to_csv(out_test, index=False)

print("\n✅ HOÀN TẤT QUÁ TRÌNH PREPROCESSING!")
print(f"📂 Dữ liệu sạch cuối cùng đã được lưu tại: {DIR}")
print(f"   1. {out_train}")
print(f"   2. {out_dev}")
print(f"   3. {out_test}")

# In mẫu kết quả cuối cùng
print("\n--- Mẫu dữ liệu cuối cùng (Final) ---")
pd.set_option('display.max_colwidth', 150)
print(df_train[['free_text', 'label_id']].head(5))

⏳ Đang đọc dữ liệu từ bước Lowercase...
⏳ Đang loại bỏ URL, Mention, Hashtag và ký tự đặc biệt...
⏳ Kiểm tra dòng rỗng sau khi clean...
⚠️ Cảnh báo: Có 152 dòng trở nên rỗng ở tập Train. (Sẽ giữ nguyên hoặc điền 'empty')

✅ HOÀN TẤT QUÁ TRÌNH PREPROCESSING!
📂 Dữ liệu sạch cuối cùng đã được lưu tại: ./UIT-ViHSD-preprocessed
   1. ./UIT-ViHSD-preprocessed\train.csv
   2. ./UIT-ViHSD-preprocessed\dev.csv
   3. ./UIT-ViHSD-preprocessed\test.csv

--- Mẫu dữ liệu cuối cùng (Final) ---
                                                            free_text  \
0                fan cứng nè ❤ ️ reaction cute coi mấy hợp_lí = ] ] ]   
1  bọn mắt híp lò_xo thụt : ) ) ) việt_nam t 10 r bọn t gọi l : ) ) )   
2                                            đậu_văn cường thằng sida   
3    côn_đồ cục_súc vô nhân_tính đề nghi vn. nhà_nước vn ban thưởng .   
4                              lý_thuyết thực_hành 1 câu_chuyện = ) )   

   label_id  
0         0  
1         2  
2         0  
3         2  
4      

# Loại bỏ các dòng rỗng ở tập train sau bước loại bỏ các ký tự đặc biệt

In [19]:
import pandas as pd
import os

# --- CẤU HÌNH ---
DIR = "./UIT-ViHSD-preprocessed"
TRAIN_FILE = os.path.join(DIR, "train.csv")

# --- HÀM XỬ LÝ ---
def remove_empty_rows(filepath):
    print(f"📂 Đang kiểm tra file: {filepath}")
    try:
        # Load dữ liệu
        df = pd.read_csv(filepath)
        
        # Thống kê trước khi xóa
        n_original = len(df)
        
        # Chuyển về string và xóa khoảng trắng thừa
        df['free_text'] = df['free_text'].fillna("").astype(str).str.strip()
        
        # Xóa các dòng rỗng (length == 0) hoặc dòng có giá trị là "nan" string
        df_clean = df[
            (df['free_text'] != '') & 
            (df['free_text'].str.lower() != 'nan') &
            (df['free_text'] != 'empty_content') # Nếu bước trước bạn có fill placeholder
        ]
        
        n_after = len(df_clean)
        n_deleted = n_original - n_after
        
        if n_deleted > 0:
            print(f"⚠️ Đã phát hiện và xóa {n_deleted} dòng rỗng.")
            # Lưu đè lại file
            df_clean.to_csv(filepath, index=False)
            print(f"✅ Đã cập nhật file: {filepath}")
        else:
            print("✅ File sạch, không có dòng rỗng nào.")
            
        print(f"   - Số lượng dòng còn lại: {n_after}")
        return df_clean

    except FileNotFoundError:
        print(f"❌ Không tìm thấy file {filepath}")

# --- THỰC THI CHO TẬP TRAIN ---
# (Bạn cũng có thể chạy cho Dev/Test nếu muốn chắc chắn)
df_train_final = remove_empty_rows(TRAIN_FILE)

📂 Đang kiểm tra file: ./UIT-ViHSD-preprocessed\train.csv
⚠️ Đã phát hiện và xóa 152 dòng rỗng.
✅ Đã cập nhật file: ./UIT-ViHSD-preprocessed\train.csv
   - Số lượng dòng còn lại: 23894
